In [1]:
import random as rd
import pandas as pd

In [2]:
# staff_list.xlsx 100人のスタッフ各員の能力を数値化したもの
# 現実ならスタッフ自身のやりたい仕事のアンケートを取る、もしくは社員やリーダーが評価する。
# その際、癒着や楽な仕事に集中しないか留意する必要がある
# 同作業NG項目も設定

# １項目の能力を0～10にしたので約5x100の労働力がある。8作業なので一つ40前後のworkが必要であるとして余剰人員には休んでもらう。
# 仕事量のクエリのtodays_work.xlsxを作成。 40 +- 20 の仕事量を設定 

todays_work=[["通常検品","バーコード検品","荷受","採寸","商品サポート","撮影アシスタント","入出庫","発送"]]
for day in range(100):
    todays_work.append([ 40 + rd.randint(-20,20) for work in range(8)])

pd.DataFrame(todays_work).to_excel("./todays_work.xlsx",header=None,index=None)


dfstaff=pd.read_excel("./staff_list.xlsx").fillna("")
dftodays_work=pd.read_excel("./todays_work.xlsx").fillna("")


In [3]:
dfstaff.head()  # 能力は0～10のランダム

,名前,通常検品,バーコード検品,荷受,採寸,商品サポート,撮影アシスタント,入出庫,発送,NG
0,佐藤,0,6,10,10,2,2,0,5,岡田
1,高橋,2,0,5,2,1,5,0,2,
2,伊藤,6,9,6,10,0,1,5,8,
3,渡辺,1,0,7,7,3,10,0,10,
4,小林,3,3,2,4,0,9,6,10,内田


In [4]:
dftodays_work.head()   # 仕事量は 40 +- 20のランダム  最大値は60*8=480で500に収まるはず

,通常検品,バーコード検品,荷受,採寸,商品サポート,撮影アシスタント,入出庫,発送
0,23,49,29,42,34,51,60,60
1,47,47,20,36,32,43,34,28
2,38,35,37,48,40,51,27,46
3,30,20,32,49,22,47,57,32
4,39,42,28,47,49,32,24,42


In [5]:
# 今回はtodays_workの仕事量を満たす、オペレーション人数の最小化を目的とする（仕事量の最大化などにすることも可能）


#  gurobiで呼び出す辞書を作成
NameWorkNg=dfstaff.keys().tolist()

dstaff={}   #スタッフの名前をキーとして能力を示したdict

for staff in dfstaff.values.tolist():
    dstaff[staff[0]]={work:value for work,value in  zip(NameWorkNg[1:],staff[1:])}
dstaff["佐藤"]



{'通常検品': 0,
 'バーコード検品': 6,
 '荷受': 10,
 '採寸': 10,
 '商品サポート': 2,
 '撮影アシスタント': 2,
 '入出庫': 0,
 '発送': 5,
 'NG': '岡田'}

In [6]:
works=NameWorkNg[1:-1]
works

['通常検品', 'バーコード検品', '荷受', '採寸', '商品サポート', '撮影アシスタント', '入出庫', '発送']

In [17]:
# いよいよgurobiで定式化して最適化する
from gurobipy import*

def opt(need_work):
    m=Model("staffing")
    m.setParam('OutputFlag',0)

    
    #スタッフをどこの業務に配置するか、バイナリー変数でとるか取らないか表現する。余剰人員という項目を追加
    d_staff_work={} 
    for work in works+["余剰人員"]:
        for name in dstaff.keys():
            d_staff_work[(name,work)]=m.addVar(vtype="B",name=name+"_"+work)
    
    m.update()
    
    # スタッフがあてがわれる業務は一つであるという制約
    for name in dstaff.keys():
        m.addConstr( quicksum( d_staff_work[(name,work)] for work in works+["余剰人員"]) ==1 )

    m.update()
    
    # スタッフ同士のNG制約
    for name in dstaff.keys():
        if dstaff[name]["NG"]:
            for work in works:
                m.addConstr( d_staff_work[(name,work)] + d_staff_work[(dstaff[name]["NG"],work)] <=1 )
                # 不等号は使えないため足して1以下になれば違う仕事のはず

    m.update()
    
    # 本日の仕事量を超えなければならないという制約    
    for work in works:
        m.addConstr( quicksum( dstaff[name][work]* d_staff_work[(name,work)] for name in dstaff.keys())>=need_work[work] )
    
    
    m.update()
    
    #目的変数(余剰人員以外の合計)の最小化
    m.setObjective( quicksum( d_staff_work[(name,work)] for name in dstaff.keys() for work in works) , GRB.MINIMIZE)
    
    #最適化
    m.optimize()
    
    kenzan=[]
    if m.Status == GRB.Status.OPTIMAL:
        for v in m.getVars():
            if v.X:
                kenzan.append(tuple(v.VarName.split("_")))
    return kenzan

#一日に必要とされる仕事のクエリに対して
for today_work in dftodays_work.values.tolist():   
    need_work={work:value for work,value in zip(works,today_work)}
    print("本日の仕事",need_work)
    
    kenzan=opt(need_work)
    kenzansum={work:0 for work in works}
    jinin=0
    for i in kenzan:
        print(i)
        name,work=i
        if work!="余剰人員":
            kenzansum[work]+=dstaff[name][work]
            jinin+=1
    print("オペレーション人数",jinin,"人")
    print(need_work)
    print(kenzansum)
    print("##########################################################################################")

本日の仕事 {'通常検品': 23, 'バーコード検品': 49, '荷受': 29, '採寸': 42, '商品サポート': 34, '撮影アシスタント': 51, '入出庫': 60, '発送': 60}
('佐々木', '通常検品')
('田中', '通常検品')
('山口', '通常検品')
('岡田', 'バーコード検品')
('石田', 'バーコード検品')
('加藤', 'バーコード検品')
('和田', 'バーコード検品')
('酒井', 'バーコード検品')
('松本', '荷受')
('長谷川', '荷受')
('岡本', '荷受')
('佐藤', '採寸')
('伊藤', '採寸')
('太田', '採寸')
('平野', '採寸')
('工藤', '採寸')
('藤井', '商品サポート')
('山下', '商品サポート')
('小島', '商品サポート')
('久保', '商品サポート')
('後藤', '撮影アシスタント')
('石井', '撮影アシスタント')
('武田', '撮影アシスタント')
('木下', '撮影アシスタント')
('中村', '撮影アシスタント')
('野口', '撮影アシスタント')
('石川', '入出庫')
('池田', '入出庫')
('小川', '入出庫')
('中川', '入出庫')
('宮本', '入出庫')
('村田', '入出庫')
('渡辺', '発送')
('小林', '発送')
('横山', '発送')
('宮崎', '発送')
('小山', '発送')
('高田', '発送')
('高橋', '余剰人員')
('吉田', '余剰人員')
('木村', '余剰人員')
('斎藤', '余剰人員')
('山崎', '余剰人員')
('森', '余剰人員')
('橋本', '余剰人員')
('前田', '余剰人員')
('坂本', '余剰人員')
('福田', '余剰人員')
('西村', '余剰人員')
('原田', '余剰人員')
('中野', '余剰人員')
('小野', '余剰人員')
('竹内', '余剰人員')
('中山', '余剰人員')
('上田', '余剰人員')
('森田', '余剰人員')
('内田', '余剰人員')
('谷口', '余剰人員')
('丸山', '余剰人

('佐々木', '通常検品')
('田中', '通常検品')
('山口', '通常検品')
('近藤', '通常検品')
('松田', '通常検品')
('今井', '通常検品')
('岡田', 'バーコード検品')
('石田', 'バーコード検品')
('加藤', 'バーコード検品')
('和田', 'バーコード検品')
('酒井', 'バーコード検品')
('松本', '荷受')
('木下', '荷受')
('中島', '荷受')
('岡本', '荷受')
('宮本', '荷受')
('佐藤', '採寸')
('伊藤', '採寸')
('林', '採寸')
('工藤', '採寸')
('藤井', '商品サポート')
('小島', '商品サポート')
('久保', '商品サポート')
('後藤', '撮影アシスタント')
('石井', '撮影アシスタント')
('武田', '撮影アシスタント')
('中村', '撮影アシスタント')
('遠藤', '撮影アシスタント')
('野口', '撮影アシスタント')
('石川', '入出庫')
('池田', '入出庫')
('小川', '入出庫')
('中川', '入出庫')
('村田', '入出庫')
('渡辺', '発送')
('横山', '発送')
('宮崎', '発送')
('高田', '発送')
('高橋', '余剰人員')
('小林', '余剰人員')
('吉田', '余剰人員')
('木村', '余剰人員')
('斎藤', '余剰人員')
('山崎', '余剰人員')
('森', '余剰人員')
('橋本', '余剰人員')
('前田', '余剰人員')
('長谷川', '余剰人員')
('坂本', '余剰人員')
('福田', '余剰人員')
('西村', '余剰人員')
('太田', '余剰人員')
('原田', '余剰人員')
('中野', '余剰人員')
('小野', '余剰人員')
('竹内', '余剰人員')
('中山', '余剰人員')
('上田', '余剰人員')
('森田', '余剰人員')
('内田', '余剰人員')
('谷口', '余剰人員')
('丸山', '余剰人員')
('大野', '余剰人員')
('菅原', '余剰人員')
('上野', '余剰人員')
('千葉', '余剰人

('石田', '通常検品')
('田中', '通常検品')
('山口', '通常検品')
('岡田', 'バーコード検品')
('大野', 'バーコード検品')
('加藤', 'バーコード検品')
('和田', 'バーコード検品')
('酒井', 'バーコード検品')
('松本', '荷受')
('石井', '荷受')
('木下', '荷受')
('中島', '荷受')
('岡本', '荷受')
('佐藤', '採寸')
('前田', '採寸')
('工藤', '採寸')
('藤井', '商品サポート')
('井上', '商品サポート')
('小島', '商品サポート')
('久保', '商品サポート')
('後藤', '撮影アシスタント')
('武田', '撮影アシスタント')
('遠藤', '撮影アシスタント')
('野口', '撮影アシスタント')
('石川', '入出庫')
('池田', '入出庫')
('小川', '入出庫')
('中川', '入出庫')
('宮本', '入出庫')
('村田', '入出庫')
('渡辺', '発送')
('小林', '発送')
('長谷川', '発送')
('横山', '発送')
('小山', '発送')
('高田', '発送')
('高橋', '余剰人員')
('伊藤', '余剰人員')
('吉田', '余剰人員')
('佐々木', '余剰人員')
('木村', '余剰人員')
('斎藤', '余剰人員')
('山崎', '余剰人員')
('森', '余剰人員')
('橋本', '余剰人員')
('坂本', '余剰人員')
('福田', '余剰人員')
('西村', '余剰人員')
('太田', '余剰人員')
('原田', '余剰人員')
('中野', '余剰人員')
('小野', '余剰人員')
('竹内', '余剰人員')
('中山', '余剰人員')
('上田', '余剰人員')
('森田', '余剰人員')
('宮崎', '余剰人員')
('内田', '余剰人員')
('谷口', '余剰人員')
('丸山', '余剰人員')
('菅原', '余剰人員')
('上野', '余剰人員')
('千葉', '余剰人員')
('増田', '余剰人員')
('平野', '余剰人員')
('渡部', '余剰人員')
('菊地

('田中', '通常検品')
('山口', '通常検品')
('高木', '通常検品')
('岡田', 'バーコード検品')
('石田', 'バーコード検品')
('加藤', 'バーコード検品')
('和田', 'バーコード検品')
('酒井', 'バーコード検品')
('村田', 'バーコード検品')
('前田', '荷受')
('長谷川', '荷受')
('中島', '荷受')
('岡本', '荷受')
('佐藤', '採寸')
('伊藤', '採寸')
('工藤', '採寸')
('藤井', '商品サポート')
('菅原', '商品サポート')
('鈴木', '商品サポート')
('山下', '商品サポート')
('小島', '商品サポート')
('久保', '商品サポート')
('後藤', '撮影アシスタント')
('石井', '撮影アシスタント')
('武田', '撮影アシスタント')
('木下', '撮影アシスタント')
('中村', '撮影アシスタント')
('野口', '撮影アシスタント')
('松本', '入出庫')
('石川', '入出庫')
('池田', '入出庫')
('小川', '入出庫')
('中川', '入出庫')
('宮本', '入出庫')
('渡辺', '発送')
('横山', '発送')
('宮崎', '発送')
('田村', '発送')
('高田', '発送')
('高橋', '余剰人員')
('小林', '余剰人員')
('吉田', '余剰人員')
('佐々木', '余剰人員')
('木村', '余剰人員')
('斎藤', '余剰人員')
('山崎', '余剰人員')
('森', '余剰人員')
('橋本', '余剰人員')
('坂本', '余剰人員')
('福田', '余剰人員')
('西村', '余剰人員')
('太田', '余剰人員')
('原田', '余剰人員')
('中野', '余剰人員')
('小野', '余剰人員')
('竹内', '余剰人員')
('中山', '余剰人員')
('上田', '余剰人員')
('森田', '余剰人員')
('内田', '余剰人員')
('谷口', '余剰人員')
('丸山', '余剰人員')
('大野', '余剰人員')
('上野', '余剰人員')
('千葉', '余剰人員')


('藤井', '商品サポート')
('菅原', '商品サポート')
('鈴木', '商品サポート')
('小島', '商品サポート')
('久保', '商品サポート')
('石井', '撮影アシスタント')
('中村', '撮影アシスタント')
('山下', '撮影アシスタント')
('遠藤', '撮影アシスタント')
('野口', '撮影アシスタント')
('石川', '入出庫')
('山本', '入出庫')
('池田', '入出庫')
('小川', '入出庫')
('横山', '発送')
('宮崎', '発送')
('上野', '発送')
('千葉', '発送')
('高田', '発送')
('河野', '発送')
('高橋', '余剰人員')
('渡辺', '余剰人員')
('小林', '余剰人員')
('吉田', '余剰人員')
('松本', '余剰人員')
('木村', '余剰人員')
('斎藤', '余剰人員')
('山崎', '余剰人員')
('森', '余剰人員')
('橋本', '余剰人員')
('前田', '余剰人員')
('後藤', '余剰人員')
('岡田', '余剰人員')
('福田', '余剰人員')
('西村', '余剰人員')
('原田', '余剰人員')
('中野', '余剰人員')
('小野', '余剰人員')
('竹内', '余剰人員')
('中山', '余剰人員')
('上田', '余剰人員')
('谷口', '余剰人員')
('丸山', '余剰人員')
('大野', '余剰人員')
('武田', '余剰人員')
('増田', '余剰人員')
('小山', '余剰人員')
('菊地', '余剰人員')
('松尾', '余剰人員')
('木下', '余剰人員')
('野村', '余剰人員')
('田中', '余剰人員')
('山田', '余剰人員')
('山口', '余剰人員')
('井上', '余剰人員')
('清水', '余剰人員')
('阿部', '余剰人員')
('藤田', '余剰人員')
('村上', '余剰人員')
('斉藤', '余剰人員')
('青木', '余剰人員')
('三浦', '余剰人員')
('藤原', '余剰人員')
('松田', '余剰人員')
('中川', '余剰人員')
('田村', '余剰人員

...

('佐々木', '通常検品')
('田中', '通常検品')
('山口', '通常検品')
('松田', '通常検品')
('石田', 'バーコード検品')
('加藤', 'バーコード検品')
('酒井', 'バーコード検品')
('松本', '荷受')
('長谷川', '荷受')
('石井', '荷受')
('木下', '荷受')
('中島', '荷受')
('岡本', '荷受')
('佐藤', '採寸')
('伊藤', '採寸')
('太田', '採寸')
('内田', '採寸')
('平野', '採寸')
('工藤', '採寸')
('藤井', '商品サポート')
('山下', '商品サポート')
('小島', '商品サポート')
('久保', '商品サポート')
('後藤', '撮影アシスタント')
('武田', '撮影アシスタント')
('石川', '入出庫')
('池田', '入出庫')
('宮本', '入出庫')
('村田', '入出庫')
('渡辺', '発送')
('横山', '発送')
('宮崎', '発送')
('小山', '発送')
('高田', '発送')
('高橋', '余剰人員')
('小林', '余剰人員')
('吉田', '余剰人員')
('木村', '余剰人員')
('斎藤', '余剰人員')
('山崎', '余剰人員')
('森', '余剰人員')
('橋本', '余剰人員')
('前田', '余剰人員')
('岡田', '余剰人員')
('坂本', '余剰人員')
('福田', '余剰人員')
('西村', '余剰人員')
('原田', '余剰人員')
('中野', '余剰人員')
('小野', '余剰人員')
('竹内', '余剰人員')
('中山', '余剰人員')
('上田', '余剰人員')
('森田', '余剰人員')
('谷口', '余剰人員')
('丸山', '余剰人員')
('大野', '余剰人員')
('菅原', '余剰人員')
('上野', '余剰人員')
('千葉', '余剰人員')
('増田', '余剰人員')
('渡部', '余剰人員')
('菊地', '余剰人員')
('松尾', '余剰人員')
('野村', '余剰人員')
('鈴木', '余剰人員')
('山本', '余剰人員')
('中村', 

('斎藤', '通常検品')
('田中', '通常検品')
('山口', '通常検品')
('松田', '通常検品')
('今井', '通常検品')
('石田', 'バーコード検品')
('野村', 'バーコード検品')
('加藤', 'バーコード検品')
('酒井', 'バーコード検品')
('長谷川', '荷受')
('武田', '荷受')
('木下', '荷受')
('中島', '荷受')
('岡本', '荷受')
('佐野', '荷受')
('佐藤', '採寸')
('伊藤', '採寸')
('太田', '採寸')
('内田', '採寸')
('平野', '採寸')
('工藤', '採寸')
('佐々木', '商品サポート')
('藤井', '商品サポート')
('久保', '商品サポート')
('後藤', '撮影アシスタント')
('柴田', '撮影アシスタント')
('野口', '撮影アシスタント')
('石川', '入出庫')
('池田', '入出庫')
('小川', '入出庫')
('中川', '入出庫')
('宮本', '入出庫')
('村田', '入出庫')
('渡辺', '発送')
('小林', '発送')
('横山', '発送')
('宮崎', '発送')
('小山', '発送')
('高田', '発送')
('高橋', '余剰人員')
('吉田', '余剰人員')
('松本', '余剰人員')
('木村', '余剰人員')
('山崎', '余剰人員')
('森', '余剰人員')
('橋本', '余剰人員')
('前田', '余剰人員')
('岡田', '余剰人員')
('石井', '余剰人員')
('坂本', '余剰人員')
('福田', '余剰人員')
('西村', '余剰人員')
('原田', '余剰人員')
('中野', '余剰人員')
('小野', '余剰人員')
('竹内', '余剰人員')
('中山', '余剰人員')
('上田', '余剰人員')
('森田', '余剰人員')
('谷口', '余剰人員')
('丸山', '余剰人員')
('大野', '余剰人員')
('菅原', '余剰人員')
('上野', '余剰人員')
('千葉', '余剰人員')
('増田', '余剰人員')
('渡部', '余剰人員')
('菊地',

('佐々木', '通常検品')
('田中', '通常検品')
('山口', '通常検品')
('近藤', '通常検品')
('松田', '通常検品')
('今井', '通常検品')
('石田', 'バーコード検品')
('加藤', 'バーコード検品')
('和田', 'バーコード検品')
('酒井', 'バーコード検品')
('松本', '荷受')
('木下', '荷受')
('中島', '荷受')
('岡本', '荷受')
('宮本', '荷受')
('佐藤', '採寸')
('伊藤', '採寸')
('太田', '採寸')
('平野', '採寸')
('工藤', '採寸')
('藤井', '商品サポート')
('鈴木', '商品サポート')
('山下', '商品サポート')
('小島', '商品サポート')
('久保', '商品サポート')
('渡辺', '撮影アシスタント')
('後藤', '撮影アシスタント')
('石井', '撮影アシスタント')
('武田', '撮影アシスタント')
('中村', '撮影アシスタント')
('野口', '撮影アシスタント')
('石川', '入出庫')
('池田', '入出庫')
('小川', '入出庫')
('村田', '入出庫')
('横山', '発送')
('宮崎', '発送')
('高田', '発送')
('高橋', '余剰人員')
('小林', '余剰人員')
('吉田', '余剰人員')
('木村', '余剰人員')
('斎藤', '余剰人員')
('山崎', '余剰人員')
('森', '余剰人員')
('橋本', '余剰人員')
('前田', '余剰人員')
('岡田', '余剰人員')
('長谷川', '余剰人員')
('坂本', '余剰人員')
('福田', '余剰人員')
('西村', '余剰人員')
('原田', '余剰人員')
('中野', '余剰人員')
('小野', '余剰人員')
('竹内', '余剰人員')
('中山', '余剰人員')
('上田', '余剰人員')
('森田', '余剰人員')
('内田', '余剰人員')
('谷口', '余剰人員')
('丸山', '余剰人員')
('大野', '余剰人員')
('菅原', '余剰人員')
('上野', '余剰人員')
('千葉', '

('山口', '通常検品')
('近藤', '通常検品')
('遠藤', '通常検品')
('松田', '通常検品')
('石田', 'バーコード検品')
('加藤', 'バーコード検品')
('酒井', 'バーコード検品')
('松本', '荷受')
('武田', '荷受')
('木下', '荷受')
('中島', '荷受')
('岡本', '荷受')
('宮本', '荷受')
('佐藤', '採寸')
('伊藤', '採寸')
('太田', '採寸')
('内田', '採寸')
('平野', '採寸')
('工藤', '採寸')
('藤井', '商品サポート')
('山下', '商品サポート')
('小島', '商品サポート')
('久保', '商品サポート')
('渡辺', '撮影アシスタント')
('後藤', '撮影アシスタント')
('石井', '撮影アシスタント')
('石川', '入出庫')
('池田', '入出庫')
('小川', '入出庫')
('中川', '入出庫')
('村田', '入出庫')
('横山', '発送')
('宮崎', '発送')
('高田', '発送')
('高橋', '余剰人員')
('小林', '余剰人員')
('吉田', '余剰人員')
('佐々木', '余剰人員')
('木村', '余剰人員')
('斎藤', '余剰人員')
('山崎', '余剰人員')
('森', '余剰人員')
('橋本', '余剰人員')
('前田', '余剰人員')
('岡田', '余剰人員')
('長谷川', '余剰人員')
('坂本', '余剰人員')
('福田', '余剰人員')
('西村', '余剰人員')
('原田', '余剰人員')
('中野', '余剰人員')
('小野', '余剰人員')
('竹内', '余剰人員')
('中山', '余剰人員')
('上田', '余剰人員')
('森田', '余剰人員')
('谷口', '余剰人員')
('丸山', '余剰人員')
('大野', '余剰人員')
('菅原', '余剰人員')
('上野', '余剰人員')
('千葉', '余剰人員')
('増田', '余剰人員')
('小山', '余剰人員')
('渡部', '余剰人員')
('菊地', '余剰人員')
('松尾', '余剰人員')


('田中', '通常検品')
('山口', '通常検品')
('松田', '通常検品')
('藤本', '通常検品')
('岡田', 'バーコード検品')
('石田', 'バーコード検品')
('加藤', 'バーコード検品')
('和田', 'バーコード検品')
('酒井', 'バーコード検品')
('村田', 'バーコード検品')
('松本', '荷受')
('武田', '荷受')
('木下', '荷受')
('中島', '荷受')
('岡本', '荷受')
('宮本', '荷受')
('佐藤', '採寸')
('伊藤', '採寸')
('平野', '採寸')
('工藤', '採寸')
('今井', '採寸')
('藤井', '商品サポート')
('渡部', '商品サポート')
('山下', '商品サポート')
('小島', '商品サポート')
('久保', '商品サポート')
('渡辺', '撮影アシスタント')
('後藤', '撮影アシスタント')
('野口', '撮影アシスタント')
('石川', '入出庫')
('井上', '入出庫')
('池田', '入出庫')
('小川', '入出庫')
('小林', '発送')
('横山', '発送')
('宮崎', '発送')
('小山', '発送')
('三浦', '発送')
('高田', '発送')
('高橋', '余剰人員')
('吉田', '余剰人員')
('佐々木', '余剰人員')
('木村', '余剰人員')
('斎藤', '余剰人員')
('山崎', '余剰人員')
('森', '余剰人員')
('橋本', '余剰人員')
('前田', '余剰人員')
('長谷川', '余剰人員')
('石井', '余剰人員')
('坂本', '余剰人員')
('福田', '余剰人員')
('西村', '余剰人員')
('太田', '余剰人員')
('原田', '余剰人員')
('中野', '余剰人員')
('小野', '余剰人員')
('竹内', '余剰人員')
('中山', '余剰人員')
('上田', '余剰人員')
('森田', '余剰人員')
('内田', '余剰人員')
('谷口', '余剰人員')
('丸山', '余剰人員')
('大野', '余剰人員')
('菅原', '余剰人員')
('上野', 

In [8]:
kenzan

[('佐々木', '通常検品'),
 ('石川', '通常検品'),
 ('後藤', '通常検品'),
 ('石田', '通常検品'),
 ('菅原', '通常検品'),
 ('遠藤', '通常検品'),
 ('青木', '通常検品'),
 ('金子', '通常検品'),
 ('村田', '通常検品'),
 ('高橋', 'バーコード検品'),
 ('渡辺', 'バーコード検品'),
 ('木村', 'バーコード検品'),
 ('宮崎', 'バーコード検品'),
 ('上野', 'バーコード検品'),
 ('千葉', 'バーコード検品'),
 ('松尾', 'バーコード検品'),
 ('近藤', 'バーコード検品'),
 ('三浦', 'バーコード検品'),
 ('今井', 'バーコード検品'),
 ('佐野', 'バーコード検品'),
 ('中山', '荷受'),
 ('丸山', '荷受'),
 ('渡部', '荷受'),
 ('田中', '荷受'),
 ('山本', '荷受'),
 ('井上', '荷受'),
 ('藤原', '荷受'),
 ('田村', '荷受'),
 ('原', '荷受'),
 ('宮本', '荷受'),
 ('藤本', '荷受'),
 ('小林', '採寸'),
 ('斎藤', '採寸'),
 ('竹内', '採寸'),
 ('森田', '採寸'),
 ('池田', '採寸'),
 ('藤田', '採寸'),
 ('村上', '採寸'),
 ('斉藤', '採寸'),
 ('安藤', '採寸'),
 ('河野', '採寸'),
 ('杉山', '採寸'),
 ('伊藤', '商品サポート'),
 ('山崎', '商品サポート'),
 ('森', '商品サポート'),
 ('福田', '商品サポート'),
 ('中野', '商品サポート'),
 ('上田', '商品サポート'),
 ('武田', '商品サポート'),
 ('小山', '商品サポート'),
 ('菊地', '商品サポート'),
 ('中村', '商品サポート'),
 ('中島', '商品サポート'),
 ('中川', '商品サポート'),
 ('小島', '商品サポート'),
 ('野口', '商品サポート'),
 ('新井', '商品サポート'),
 ('吉田', '撮影アシ

In [9]:
# 今後の課題

# 時給制なので、一時間ごとに再計算？もしくは時系列も考慮するなど。
# 確実に来る人員と当日バックレや病欠が起きる人員でモデルを分ける必要があるかもしれない。
# 週四以上の契約、週三以下の契約などの制約条件もある。
# 現実はもっと複雑なはず。